<a href="https://colab.research.google.com/github/tanvimagdum/Customer-Feedback-Analysis-and-Summarization/blob/main/MiniProject_IDMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install kagglehub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("cynthiarempel/amazon-us-customer-reviews-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/omennoob/.cache/kagglehub/datasets/cynthiarempel/amazon-us-customer-reviews-dataset/versions/9


In [3]:
# Data Loading and Setup
path = "/Users/omennoob/.cache/kagglehub/datasets/cynthiarempel/amazon-us-customer-reviews-dataset/versions/9"
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonReviewsAnalysis").config("spark.driver.memory", "16g").getOrCreate()
df = spark.read.csv(path, sep="\t", header=True, inferSchema=True)
df.printSchema()

24/11/23 23:32:07 WARN Utils: Your hostname, Rohits-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.24 instead (on interface en0)
24/11/23 23:32:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/23 23:32:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/23 23:32:20 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [7]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [15]:
# Data Preprocessing

from pyspark.sql.functions import col, lower, regexp_replace, length, length, trim, when, size
from pyspark.ml.feature import Tokenizer, StopWordsRemover

# Convert text to lowercase and remove punctuation
df_cleaned = df.withColumn("review_body", lower(col("review_body")))
df_cleaned = df_cleaned.withColumn("review_body", regexp_replace(col("review_body"), "[^a-zA-Z0-9\\s]", ""))

# Tokenization
tokenizer = Tokenizer(inputCol="review_body", outputCol="tokens")
df_tokenized = tokenizer.transform(df_cleaned)

# Remove stop words
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")
df_filtered = remover.transform(df_tokenized)

# Filter Reviews by Ratings
df_positive = df_filtered.filter(col("star_rating") >= 4)
df_negative = df_filtered.filter(col("star_rating") <= 2)

# Filter out NULL and empty values in review_body
df_filtered = df_filtered.filter(col("review_body").isNotNull())
df_filtered = df_filtered.filter(length(trim(col("review_body"))) > 0)

# Add Sentiment Column
df_filtered = df_filtered.withColumn(
    "sentiment",
    when(col("star_rating") >= 4, "positive")
    .when(col("star_rating") <= 2, "negative")
    .otherwise("neutral")
)

# Remove rows with empty filtered_tokens
df_filtered = df_filtered.filter(size(col("filtered_tokens")) > 0)


In [19]:
# from pyspark.sql.functions import col, length, trim, when, size
# from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer

# # Step 1: Filter out NULL and empty values in review_body
# df_filtered = df_filtered.filter(col("review_body").isNotNull())
# df_filtered = df_filtered.filter(length(trim(col("review_body"))) > 0)

# # Step 2: Add Sentiment Column
# df_filtered = df_filtered.withColumn(
#     "sentiment",
#     when(col("star_rating") >= 4, "positive")
#     .when(col("star_rating") <= 2, "negative")
#     .otherwise("neutral")
# )

# # Step 3: Tokenize and Remove Stop Words
# tokenizer = Tokenizer(inputCol="review_body", outputCol="tokens_new")  # Changed output column name
# df_tokenized = tokenizer.transform(df_filtered)

# remover = StopWordsRemover(inputCol="tokens_new", outputCol="filteredTokens")
# df_filtered = remover.transform(df_tokenized)

# # Step 4: Remove rows with empty filtered_tokens
# df_filtered = df_filtered.filter(size(col("filteredTokens")) > 0)

# # Step 5: Fit CountVectorizer
# cv = CountVectorizer(inputCol="filteredTokens", outputCol="key_phrases", vocabSize=50)
# model = cv.fit(df_filtered)
# df_with_phrases = model.transform(df_filtered)


In [23]:
from pyspark.sql.functions import col, lower, regexp_replace, when, size, concat_ws, round
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Data Loading (Assuming df_cleaned is already created as per the provided script)

# Step 1: Split Dataset
train_data, test_data = df_filtered.randomSplit([0.8, 0.2], seed=42)  # 80-20 split

# Step 2: Fit CountVectorizer on Training Data
cv = CountVectorizer(inputCol="filtered_tokens", outputCol="key_phrases", vocabSize=50)
cv_model = cv.fit(train_data)

# Transform both training and testing data
train_transformed = cv_model.transform(train_data)
test_transformed = cv_model.transform(test_data)

# Step 3: Train a Classifier
# Add numeric labels for sentiment
train_transformed = train_transformed.withColumn(
    "label",
    when(col("sentiment") == "positive", 1.0)
    .when(col("sentiment") == "negative", 0.0)
    .otherwise(2.0)  # Neutral
)

test_transformed = test_transformed.withColumn(
    "label",
    when(col("sentiment") == "positive", 1.0)
    .when(col("sentiment") == "negative", 0.0)
    .otherwise(2.0)  # Neutral
)

# Train a Logistic Regression Model
lr = LogisticRegression(featuresCol="key_phrases", labelCol="label")
lr_model = lr.fit(train_transformed)

# Step 4: Make Predictions
predictions = lr_model.transform(test_transformed)

# Step 5: Evaluate the Model
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy"
)
accuracy = evaluator.evaluate(predictions)

# Print Predictions and Accuracy
predictions.select("product_category", "sentiment", "prediction").show(10, truncate=False)
print(f"Model Accuracy: {accuracy:.2f}")


24/11/24 00:51:22 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/11/24 00:51:22 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


+----------------------+---------+----------+
|product_category      |sentiment|prediction|
+----------------------+---------+----------+
|Digital_Music_Purchase|positive |1.0       |
|Digital_Music_Purchase|positive |1.0       |
|Digital_Music_Purchase|positive |1.0       |
|Digital_Music_Purchase|positive |1.0       |
|Digital_Music_Purchase|positive |1.0       |
|Digital_Music_Purchase|positive |1.0       |
|Digital_Music_Purchase|positive |1.0       |
|Digital_Music_Purchase|positive |1.0       |
|Digital_Music_Purchase|positive |1.0       |
|Digital_Music_Purchase|positive |1.0       |
+----------------------+---------+----------+
only showing top 10 rows

Model Accuracy: 0.78


In [29]:
print(f"Model Accuracy: {accuracy:.2f}")


Model Accuracy: 0.78


In [33]:
from pyspark.sql.functions import concat_ws

# Combine tokens back to sentences
df_filtered = df_filtered.withColumn("summary", concat_ws(" ", col("filtered_tokens")))

# Summaries for each sentiment
df_positive_summary = df_filtered.filter(col("sentiment") == "positive").select("summary").show(10, truncate=False)
df_negative_summary = df_filtered.filter(col("sentiment") == "negative").select("summary").show(10, truncate=False)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|summary                                                                                                                                                          |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|great  rendition great  song                                                                                                                                     |
|good music listen                                                                                                                                                |
|good original needed hear parts easily heard one                                                                                                                 |
|time worship su

In [35]:
insights = df_filtered.groupBy("product_category", "sentiment").count()
insights.show()

+--------------------+---------+-------+
|    product_category|sentiment|  count|
+--------------------+---------+-------+
|Digital_Music_Pur...| negative|  79434|
|Digital_Music_Pur...|  neutral|  67580|
|Digital_Music_Pur...| positive|1646047|
|              Sports|  neutral| 380493|
|              Sports| positive|3873876|
|              Sports| negative| 592585|
|             Watches| negative| 138412|
|             Watches|  neutral|  79799|
|             Watches| positive| 752009|
|            Outdoors| positive|1852733|
|            Outdoors| negative| 271118|
|            Outdoors|  neutral| 179295|
|          Automotive| negative| 447339|
|          Automotive| positive|2821247|
|          Automotive|  neutral| 239661|
|             Grocery| positive|1953551|
|             Grocery| negative| 285133|
|             Grocery|  neutral| 161362|
|     Office Products|  neutral| 193883|
|     Office Products| positive|2001881|
+--------------------+---------+-------+
only showing top

In [36]:
from pyspark.sql.functions import col, round

# Group by product_category and calculate total reviews
total_reviews = df_filtered.groupBy("product_category").count().withColumnRenamed("count", "total_count")

# Join insights with total_reviews on product_category
insights_with_totals = insights.join(total_reviews, on="product_category", how="inner")

# Add the percentage column
sentiment_stats = insights_with_totals.withColumn(
    "percentage",
    round((col("count") / col("total_count")) * 100, 2)
)

# Show the result
sentiment_stats.show()


+--------------------+---------+-------+-----------+----------+
|    product_category|sentiment|  count|total_count|percentage|
+--------------------+---------+-------+-----------+----------+
|Digital_Music_Pur...| positive|1646047|    1793061|      91.8|
|Digital_Music_Pur...|  neutral|  67580|    1793061|      3.77|
|Digital_Music_Pur...| negative|  79434|    1793061|      4.43|
|              Sports| negative| 592585|    4846954|     12.23|
|              Sports| positive|3873876|    4846954|     79.92|
|              Sports|  neutral| 380493|    4846954|      7.85|
|             Watches| positive| 752009|     970220|     77.51|
|             Watches|  neutral|  79799|     970220|      8.22|
|             Watches| negative| 138412|     970220|     14.27|
|            Outdoors|  neutral| 179295|    2303146|      7.78|
|            Outdoors| negative| 271118|    2303146|     11.77|
|            Outdoors| positive|1852733|    2303146|     80.44|
|          Automotive|  neutral| 239661|